# Betas v2

## Import libraries

In [1]:
first_folder_path = '../../../'

In [2]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Load data

In [3]:
rutas_edges_var = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/03_rutas_en_edges/rutas_edges_var.gpkg")

# Show
print(f"{len(rutas_edges_var.code.unique())} unique codes.")
print(rutas_edges_var.crs)
print(rutas_edges_var.shape)
rutas_edges_var.head(2)

201 unique codes.
EPSG:32613
(1882, 21)


,code,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,...,pobtot_v,total_pobtot,average_population,dens_u,dens_v,average_density,interdens_u,interdens_v,average_intersdens,geometry
0,JC1V008,66819495228861751,66820751228869398,0,0.012849,0.15356,1.726807,316.602353,0.0,1.0,...,70.0,172.0,86.0,65.5488,50.632364,58.090582,12.761922,12.153251,12.457586,"MULTILINESTRING ((668194.953 2288617.514, 6682..."
1,MeV1V006,66819495228861751,66820751228869398,0,0.012849,0.15356,1.726807,316.602353,0.0,1.0,...,70.0,172.0,86.0,65.5488,50.632364,58.090582,12.761922,12.153251,12.457586,"MULTILINESTRING ((668194.953 2288617.514, 6682..."


In [4]:
final_routes = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/final_routes.gpkg")

# Show
print(f"{len(final_routes.Código.unique())} unique codes.")
print(final_routes.crs)
print(final_routes.shape)
final_routes.head(2)

201 unique codes.
EPSG:32613
(201, 70)


,Código,layer,path,field_1,field_2,field_3,field_4,field_5,field_6,field_7,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry
0,DP3V002,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,252,9/19/24 18:40:07,9/19/24 18:46:58,anonymous,None,None,DP,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,"LINESTRING (660511.087 2283608.716, 660443.453..."
1,DP3V005,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,255,9/19/24 18:50:40,9/19/24 18:57:14,anonymous,None,None,DP,...,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,"LINESTRING (660562.586 2283275.252, 660591.611..."


## Merge data

In [7]:
merged = pd.merge(rutas_edges_var,final_routes, left_on='code',right_on='Código')

print(f"{len(merged.code.unique())} unique codes.")
print(merged.shape)
merged.head(2)

201 unique codes.
(1882, 91)


,code,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry_y
0,JC1V008,66819495228861751,66820751228869398,0,0.012849,0.153560,1.726807,316.602353,0.0,1.0,...,De acuerdo,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,"LINESTRING (667358.311 2288494.110, 667701.720..."
1,JC1V008,66820751228869398,66821647228874845,0,0.000000,0.159422,1.487973,272.033757,0.0,1.0,...,De acuerdo,De acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,"LINESTRING (667358.311 2288494.110, 667701.720..."


## Relate variables and betas by polygon